In [1]:
# installing necessary packages
!pip install beautifulsoup4
!pip install lxml

In [2]:
# step 1 - Reading the data from xml file and picking the filed inside <object> tags

from bs4 import BeautifulSoup

with open('example.xml', 'r') as f:
    data = f.read()

Bs_data = BeautifulSoup(data, "xml")

objects = Bs_data.find_all('object')
 
print(objects[0].find('name').text)

Canvas


In [3]:
# step 2 - making a Node class to represent a Tree Node with same properties as there for attributes of object tag in xml 
# file
class Node:
    
    def __init__(self,name,pose,truncated,xmin,xmax,ymin,ymax,childeren):
        self.name = name
        self.pose = pose
        self.truncated = truncated
        self.xmin = int(xmin)
        self.xmax = int(xmax)
        self.ymin = int(ymin)
        self.ymax = int(ymax)
        self.area = (self.ymax - self.ymin)*(self.xmax - self.xmin)
        self.children = childeren or []
    def __lt__(self, other):
         return self.area > other.area
    
    

In [4]:
# Step 3 - creating object of Node class corresponding to object tag and storing them in a list
node = []

for obj in objects:
    tmp_node = Node(obj.find('name').text, obj.find('pose').text, obj.find('truncated').text,
               obj.find('xmin').text, obj.find('xmax').text, obj.find('ymin').text, obj.find('ymax').text,[])
    node.append(tmp_node)

In [5]:
 # step 4 -sorting the object based on the area of bounding box (bounding box whose area is most will come first and so on)
node.sort()

In [6]:
# step 5 - defining a funtion which basically check whether a object(Node) is inside a object(Node) based on 
# properties of object tag like (xmin,xmax,ymin,ymax)

def isContained(parent,child):
    if parent.xmin <= child.xmin and parent.xmax >= child.xmax and parent.ymin <= child.ymin and parent.ymax >=child.ymax:
        return True
    return False

In [7]:
# step 6 - creating parent child relationship for all the nodes in the tree
parent = [-1]*len(node)

parent[0] = node[0]

for index in range(1,len(node)):
    
    for j in range(index-1,-1,-1):
        
        if isContained(node[j], node[index]):
            parent[index] = node[j]
            break;
            
    



In [8]:
# step 7 - Linking parent with child node

for i in range(len(parent)-1,0,-1):
    parent[i].children.append(node[i])

In [9]:
# step 8 -Doing a BFS traversal over the tree
def printTree(node):
    queue = []
 
    # Enqueue Root and initialize height
    queue.append(node)
 
    while(len(queue) > 0):
        
        
        level_length = len(queue);
        
        while level_length > 0:
            node = queue.pop(0)
            print(node.name, end = " ")
            for child in node.children:
                queue.append(child)
            level_length= level_length -1;
        print()
        print('********************')
    

In [10]:
printTree(node[0])

Canvas 
********************
Header Side Bar Grid 
********************
Button Button Button Button sidebar content sidebar content sidebar content sidebar content Card Card 
********************
Button Image Button Image 
********************


In [11]:
# step 9 - I have hashed the preorder traversal of the representation of entire subtree and then check their number of occurence to
# to find similar componenets
tree_map = {}

repeated_node = []

def similarComponenets(node,path):
    
    if not node:
        return '#'
    
    path += node.name
    
    print(path)
    
    for child in node.children:
        path += similarComponenets(child, '')
    
    if path in tree_map:
        tree_map[path]+=1;
        
        if tree_map[path] == 2:
            repeated_node.append(node)
    else:
        tree_map[path] = 1
    
    return path
            
            
        
    
    

In [12]:
similarComponenets(node[0],'')

Canvas
Header
Button
Button
Button
Button
Side Bar
sidebar content
sidebar content
sidebar content
sidebar content
Grid
Card
Button
Image
Card
Button
Image


'CanvasHeaderButtonButtonButtonButtonSide Barsidebar contentsidebar contentsidebar contentsidebar contentGridCardButtonImageCardButtonImage'

In [13]:
for obj in repeated_node:
    printTree(obj)
    print("----------------")

Button 
********************
----------------
sidebar content 
********************
----------------
Image 
********************
----------------
Card 
********************
Button Image 
********************
----------------
